# Question word prediction

> Group 12: Tristan Perrot & Romain Darous

Task is to train and evaluate a QWP model using any available QA-corpus, for instance, the [SQuAD corpus](https://rajpurkar.github.io/SQuAD-explorer/).


In [1]:
import json
import math
import os

import matplotlib.pyplot as plt
import nltk
import pandas as pd
import requests
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm.notebook import tqdm, trange

In [2]:
for i in range(torch.cuda.device_count()):
    print(torch.cuda.get_device_properties(i).name)

device = torch.device("cuda" if torch.cuda.is_available(
) else "mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

## Data


In [3]:
data_dir = 'data'

In [4]:
if data_dir not in os.listdir():
    os.mkdir(data_dir)

if "squad_train.json" not in os.listdir(data_dir):
    # Download data at https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
    res = requests.get(
        "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json")
    data = json.loads(res.text)

    # Save data to file
    with open(data_dir + "/squad_train.json", "w") as f:
        json.dump(data, f)

with open(data_dir + "/squad_train.json", "r") as f:
    data = json.load(f)

# Extract answer text and question text
answers = []
questions = []
for article in data["data"]:
    for paragraph in article["paragraphs"]:
        for qa in paragraph["qas"]:
            if qa["is_impossible"]:
                continue
            answers.append(qa["answers"][0]["text"])
            questions.append(qa["question"])

print("Number of questions:", len(questions))

# Print some examples
for i in range(5):
    print()
    print("Question:", questions[i])
    print("Answer:", answers[i])

Number of questions: 86821

Question: When did Beyonce start becoming popular?
Answer: in the late 1990s

Question: What areas did Beyonce compete in when she was growing up?
Answer: singing and dancing

Question: When did Beyonce leave Destiny's Child and become a solo singer?
Answer: 2003

Question: In what city and state did Beyonce  grow up? 
Answer: Houston, Texas

Question: In which decade did Beyonce become famous?
Answer: late 1990s


In [5]:
# Tokenize questions
tokenized_questions = [nltk.word_tokenize(q) for q in questions]

# Tokenize answers
tokenized_answers = [nltk.word_tokenize(a) for a in answers]

for i in range(5):
    print()
    print("Question:", tokenized_questions[i])
    print("Answer:", tokenized_answers[i])


Question: ['When', 'did', 'Beyonce', 'start', 'becoming', 'popular', '?']
Answer: ['in', 'the', 'late', '1990s']

Question: ['What', 'areas', 'did', 'Beyonce', 'compete', 'in', 'when', 'she', 'was', 'growing', 'up', '?']
Answer: ['singing', 'and', 'dancing']

Question: ['When', 'did', 'Beyonce', 'leave', 'Destiny', "'s", 'Child', 'and', 'become', 'a', 'solo', 'singer', '?']
Answer: ['2003']

Question: ['In', 'what', 'city', 'and', 'state', 'did', 'Beyonce', 'grow', 'up', '?']
Answer: ['Houston', ',', 'Texas']

Question: ['In', 'which', 'decade', 'did', 'Beyonce', 'become', 'famous', '?']
Answer: ['late', '1990s']


In [6]:
# Merge questions and answers
merged = [q + a for q, a in zip(tokenized_questions, tokenized_answers)]

for i in range(5):
    print()
    print("Merged:", merged[i])


Merged: ['When', 'did', 'Beyonce', 'start', 'becoming', 'popular', '?', 'in', 'the', 'late', '1990s']

Merged: ['What', 'areas', 'did', 'Beyonce', 'compete', 'in', 'when', 'she', 'was', 'growing', 'up', '?', 'singing', 'and', 'dancing']

Merged: ['When', 'did', 'Beyonce', 'leave', 'Destiny', "'s", 'Child', 'and', 'become', 'a', 'solo', 'singer', '?', '2003']

Merged: ['In', 'what', 'city', 'and', 'state', 'did', 'Beyonce', 'grow', 'up', '?', 'Houston', ',', 'Texas']

Merged: ['In', 'which', 'decade', 'did', 'Beyonce', 'become', 'famous', '?', 'late', '1990s']


In [7]:
# Create vocabulary
vocab = set()
for m in merged:
    vocab.update(m)

vocab = list(vocab)

# Add "<qw>" to vocabulary
vocab.append("<qw>")
vocab.append("<pad>")

# Create word to index and index to word mappings
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for i, word in enumerate(vocab)}

In [12]:
words_to_hide = 1

hidden_merged = []

for m in merged:
    hidden = m.copy()
    hidden[:words_to_hide] = ["<qw>"] * words_to_hide
    hidden_merged.append(hidden)

for i in range(5):
    print()
    print("Merged:", merged[i])
    print("Hidden:", hidden_merged[i])

interr_words = set()

for questions in merged:
    interr_words.update(questions[:words_to_hide])

print()
print(interr_words)
print(len(interr_words))


Merged: ['When', 'did', 'Beyonce', 'start', 'becoming', 'popular', '?', 'in', 'the', 'late', '1990s']
Hidden: ['<qw>', 'did', 'Beyonce', 'start', 'becoming', 'popular', '?', 'in', 'the', 'late', '1990s']

Merged: ['What', 'areas', 'did', 'Beyonce', 'compete', 'in', 'when', 'she', 'was', 'growing', 'up', '?', 'singing', 'and', 'dancing']
Hidden: ['<qw>', 'areas', 'did', 'Beyonce', 'compete', 'in', 'when', 'she', 'was', 'growing', 'up', '?', 'singing', 'and', 'dancing']

Merged: ['When', 'did', 'Beyonce', 'leave', 'Destiny', "'s", 'Child', 'and', 'become', 'a', 'solo', 'singer', '?', '2003']
Hidden: ['<qw>', 'did', 'Beyonce', 'leave', 'Destiny', "'s", 'Child', 'and', 'become', 'a', 'solo', 'singer', '?', '2003']

Merged: ['In', 'what', 'city', 'and', 'state', 'did', 'Beyonce', 'grow', 'up', '?', 'Houston', ',', 'Texas']
Hidden: ['<qw>', 'what', 'city', 'and', 'state', 'did', 'Beyonce', 'grow', 'up', '?', 'Houston', ',', 'Texas']

Merged: ['In', 'which', 'decade', 'did', 'Beyonce', 'beco

In [ ]:
words_to_pred = [
    ['What'],
    ['When'],
    ['Where'],
    ['Who'],
    ['Whom'],
    ['In', 'what'],
    ['In', 'which'],
    ['Which'],
    ['How', 'much'],
    ['How', 'muany'],

]

In [ ]:
restricted_merged = []

for questions in merged

In [9]:
x_words = [merged_question[::-1] for merged_question in merged]
y_words = [hidden_answer[::-1] for hidden_answer in hidden_merged]

for i in range(5):
    print()
    print("X_words:", x_words[i])
    print("Y_words:", y_words[i])


X_words: ['1990s', 'late', 'the', 'in', '?', 'popular', 'becoming', 'start', 'Beyonce', 'did', 'When']
Y_words: ['1990s', 'late', 'the', 'in', '?', 'popular', 'becoming', 'start', 'Beyonce', 'did', '<qw>']

X_words: ['dancing', 'and', 'singing', '?', 'up', 'growing', 'was', 'she', 'when', 'in', 'compete', 'Beyonce', 'did', 'areas', 'What']
Y_words: ['dancing', 'and', 'singing', '?', 'up', 'growing', 'was', 'she', 'when', 'in', 'compete', 'Beyonce', 'did', 'areas', '<qw>']

X_words: ['2003', '?', 'singer', 'solo', 'a', 'become', 'and', 'Child', "'s", 'Destiny', 'leave', 'Beyonce', 'did', 'When']
Y_words: ['2003', '?', 'singer', 'solo', 'a', 'become', 'and', 'Child', "'s", 'Destiny', 'leave', 'Beyonce', 'did', '<qw>']

X_words: ['Texas', ',', 'Houston', '?', 'up', 'grow', 'Beyonce', 'did', 'state', 'and', 'city', 'what', 'In']
Y_words: ['Texas', ',', 'Houston', '?', 'up', 'grow', 'Beyonce', 'did', 'state', 'and', 'city', 'what', '<qw>']

X_words: ['1990s', 'late', '?', 'famous', 'become

In [10]:
# Convert words to indices
def words_to_indices(words):
    return [word_to_idx[word] for word in words]


x = [words_to_indices(words) for words in x_words]
y = [words_to_indices(words) for words in y_words]

for i in range(5):
    print()
    print("X:", x[i])
    print("Y:", y[i])


X: [21544, 48170, 49668, 31874, 31149, 3602, 14896, 1835, 21150, 12618, 65083]
Y: [21544, 48170, 49668, 31874, 31149, 3602, 14896, 1835, 21150, 12618, 67298]

X: [20898, 61937, 46659, 31149, 41438, 30710, 15960, 33499, 30236, 31874, 45462, 21150, 12618, 29146, 25588]
Y: [20898, 61937, 46659, 31149, 41438, 30710, 15960, 33499, 30236, 31874, 45462, 21150, 12618, 29146, 67298]

X: [23869, 31149, 41559, 66569, 59427, 4070, 61937, 16401, 45391, 41880, 61146, 21150, 12618, 65083]
Y: [23869, 31149, 41559, 66569, 59427, 4070, 61937, 16401, 45391, 41880, 61146, 21150, 12618, 67298]

X: [17821, 13572, 3213, 31149, 41438, 16864, 21150, 12618, 22201, 61937, 43286, 43050, 49514]
Y: [17821, 13572, 3213, 31149, 41438, 16864, 21150, 12618, 22201, 61937, 43286, 43050, 67298]

X: [21544, 48170, 31149, 31935, 4070, 21150, 12618, 1549, 66352, 49514]
Y: [21544, 48170, 31149, 31935, 4070, 21150, 12618, 1549, 66352, 67298]


In [11]:
# Pad sequences
max_len = max([len(words) for words in x])

for i in range(len(x)):
    x[i] += [word_to_idx["<pad>"]] * (max_len - len(x[i]))
    y[i] += [word_to_idx["<pad>"]] * (max_len - len(y[i]))

for i in range(5):
    print()
    print("X:", x[i])
    print("Y:", y[i])


X: [21544, 48170, 49668, 31874, 31149, 3602, 14896, 1835, 21150, 12618, 65083, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299]
Y: [21544, 48170, 49668, 31874, 31149, 3602, 14896, 1835, 21150, 12618, 67298, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299, 67299]

X: [20898, 61937, 46659, 31149, 41438, 30710, 15960, 33499, 30236, 31874, 45462, 21150, 12618, 29146, 25588, 672

In [12]:
# Convert to tensors
x = torch.tensor(x).to(device)
y = torch.tensor(y).to(device)

In [13]:
# Example of a training pair (x, y) words
i = 0
print("X:", [idx_to_word[idx.item()] for idx in x[i]])
print("Y:", [idx_to_word[idx.item()] for idx in y[i]])

X: ['1990s', 'late', 'the', 'in', '?', 'popular', 'becoming', 'start', 'Beyonce', 'did', 'When', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
Y: ['1990s', 'late', 'the', 'in', '?', 'popular', 'becoming', 'start', 'Beyonce', 'did', '<qw>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad

## Training

We will now our model to predict the first or the two first words of the question given the rest of the question and the answer. We will use a transformer model to do so.


In [14]:
# Train val test split
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.5)

print("Train size:", len(x_train))
print("Val size:", len(x_val))
print("Test size:", len(x_test))

# Create dataset


class QuestionDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]


train_dataset = QuestionDataset(x_train, y_train)
val_dataset = QuestionDataset(x_val, y_val)
test_dataset = QuestionDataset(x_test, y_test)

# Create dataloaders
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Train size: 69456
Val size: 8682
Test size: 8683


### LSTM simple test model


In [15]:
# Define model
class QuestionWordPredictor(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(QuestionWordPredictor, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = self.fc(x)
        return x


# Initialize model
vocab_size = len(vocab)
embedding_dim = 100
hidden_dim = 100

model = QuestionWordPredictor(vocab_size, embedding_dim, hidden_dim).to(device)

# Define optimizer
optimizer = optim.Adam(model.parameters())

# Define loss function
criterion = nn.CrossEntropyLoss()

# Train model
num_epochs = 10

for epoch in trange(num_epochs):
    model.train()
    train_loss = 0
    for x_batch, y_batch in tqdm(train_loader):
        optimizer.zero_grad()
        y_pred = model(x_batch)
        loss = criterion(y_pred.view(-1, vocab_size), y_batch.view(-1))
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for x_batch, y_batch in val_loader:
            y_pred = model(x_batch)
            loss = criterion(y_pred.view(-1, vocab_size), y_batch.view(-1))
            val_loss += loss.item()
        val_loss /= len(val_loader)

    print(f"Epoch {
          epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2171 [00:00<?, ?it/s]

Epoch 1/10, Train Loss: 0.9235, Val Loss: 0.3803


  0%|          | 0/2171 [00:00<?, ?it/s]

Epoch 2/10, Train Loss: 0.2385, Val Loss: 0.1969


...

Epoch 8/10, Train Loss: 0.0014, Val Loss: 0.1114


  0%|          | 0/2171 [00:00<?, ?it/s]

Epoch 9/10, Train Loss: 0.0010, Val Loss: 0.1103


  0%|          | 0/2171 [00:00<?, ?it/s]

Epoch 10/10, Train Loss: 0.0007, Val Loss: 0.1099


In [16]:
# Evaluate model
model.eval()
test_loss = 0
with torch.no_grad():
    for x_batch, y_batch in test_loader:
        y_pred = model(x_batch)
        loss = criterion(y_pred.view(-1, vocab_size), y_batch.view(-1))
        test_loss += loss.item()
    test_loss /= len(test_loader)

print(f"Test Loss: {test_loss:.4f}")

def accuracy(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x_batch, y_batch in loader:
            y_pred = model(x_batch)
            y_pred = y_pred.argmax(dim=-1)
            correct += (y_pred == y_batch).sum().item()
            total += y_batch.numel()
    return correct / total


train_accuracy = accuracy(model, train_loader)
val_accuracy = accuracy(model, val_loader)
test_accuracy = accuracy(model, test_loader)

print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Val Accuracy: {val_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Loss: 0.1114
Train Accuracy: 0.9999
Val Accuracy: 0.9925
Test Accuracy: 0.9923


In [17]:
if not os.path.exists("models"):
    os.mkdir("models")

# Save model
torch.save(model.state_dict(), "models/question_word_predictor.pth")

# Load model
model = QuestionWordPredictor(vocab_size, embedding_dim, hidden_dim).to(device)
model.load_state_dict(torch.load("models/question_word_predictor.pth"))

<All keys matched successfully>

In [19]:
# Predict
def predict(model, x):
    model.eval()
    with torch.no_grad():
        y_pred = model(x)
        y_pred = y_pred.argmax(dim=-1)
    return y_pred


# Example of a training pair (x, y) words
i = 0
x_i = x_train[i].unsqueeze(0)
y_pred = predict(model, x_i)
print("X:", [idx_to_word[idx.item()] for idx in x_i[0]])
print("Y:", [idx_to_word[idx.item()] for idx in y_train[i]])
print("Y_pred:", [idx_to_word[idx.item()] for idx in y_pred[0]])

X: ['1963', '?', 'die', 'XXIII', 'John', 'Pope', 'did', 'year', 'what', 'In', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
Y: ['1963', '?', 'die', 'XXIII', 'John', 'Pope', 'did', 'year', 'what', '<qw>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad